# Algoritmos Genéticos

## Estrutura básica do AG

```
1. t = 0
2. Inicializar a população inicial P_0
3. Enquanto critério de parada == falso
   a. Avaliar a população(Pt)
   b. P' = Selecionar pais(Pt)
   c. F = Aplicar recombinação e mutação(P')
   d. Avaliar a população(F)
   e. Pt+1 = Selecionar sobreviventes(Pt + F)
   f. t = t + 1
```

In [1]:
# Importa a biblioteca para criação de classes de dados
from dataclasses import dataclass

# Importação dos tipos de dados para o type hint
from typing import Callable

# Importação da biblioteca de números aleatórios
from random import randint
from random import sample
from random import random

# Importação da biblioteca de impressão bonita
from pprint import pprint


In [2]:
for i in range(10):
    print(randint(0, 1))

0
1
0
1
1
0
1
1
1
1


In [3]:
sample([1, 2, 3, 4, 5, 6], 3)

[4, 2, 5]

In [4]:
@dataclass 
class Cromossomo:
    dados: list[int]
    fitness: float = 0

In [5]:
@dataclass
class Config:
    tam_cromossomo: int
    tam_populacao: int
    fitness: Callable[[Cromossomo], float]

    selecionar_pais: Callable[[list[Cromossomo]], list[tuple[Cromossomo, Cromossomo]]]

    aplicar_cruzamento: Callable[[list[tuple[Cromossomo, Cromossomo]]], list[Cromossomo]]
    
    aplicar_mutacao: Callable[[list[Cromossomo], float], None]
    taxa_mutacao: float

    selecionar_sobreviventes: Callable[[list[Cromossomo]], list[Cromossomo]]


    

## Inicialização da população

In [6]:
def inicializar_populacao(tam_populacao, tam_cromossomo) -> list[Cromossomo]:
    P0 = []
    for i in range(tam_populacao):
        cromossomo = Cromossomo([randint(0, 1) for j in range(tam_cromossomo)])
        P0.append(cromossomo)
    return P0

In [7]:
print(inicializar_populacao(4, 8))

[Cromossomo(dados=[1, 1, 0, 1, 1, 0, 0, 0], fitness=0), Cromossomo(dados=[0, 1, 0, 0, 1, 1, 1, 1], fitness=0), Cromossomo(dados=[1, 0, 1, 1, 0, 0, 1, 0], fitness=0), Cromossomo(dados=[0, 1, 1, 0, 1, 0, 0, 0], fitness=0)]


## Seleção dos pais

In [8]:
# Seleção por torneio - com reposição
# - Seleciona k pais e o tiver melhor fitness é selecionado para formar casal
def torneio(populacao: list[Cromossomo], tam_torneio: int = 3) -> list[tuple[Cromossomo, Cromossomo]]:
    casais = []

    for i in range(len(populacao) // 2):

        torneio1 = sample(populacao, tam_torneio * 2)
        pai1 = sorted(torneio1, key=lambda x: x.fitness, reverse=True)[0] 
        
        torneio2 = sample(populacao, tam_torneio)
        pai2 = sorted(torneio2, key=lambda x: x.fitness, reverse=True)[0] 

        # TODO: evitar que o selecionado no pai1 seja selecionado como pai2

        casais.append((pai1, pai2))
    
    return casais

In [9]:
## Crossover

In [10]:
def crossover_1_corte(casais):
    filhos = []

    for par in casais:

        # Desempacota os pais
        pai1 = par[0].dados
        pai2 = par[1].dados

        # Sorteia o ponto de corte
        corte = randint(1, len(pai1) - 1)

        filho1 = pai1[:corte] + pai2[corte:] 
        filho2 = pai2[:corte] + pai1[corte:] 

        filhos.append(Cromossomo(filho1))
        filhos.append(Cromossomo(filho2))

    return filhos

In [11]:
## Mutação

In [18]:
def mutacao(cromossomos: list[Cromossomo], taxa_mutacao: float):
    for cromossomo in cromossomos:
        for i, c in enumerate(cromossomo.dados):
            if random() < taxa_mutacao:
                if c == 1:
                    cromossomo.dados[i] = 0
                else:
                    cromossomo.dados[i] = 1
                        
    

In [13]:
## Seleção dos sobreviventes

In [14]:
def elitismo(populacao):
    populacao.sort(key=lambda x: x.fitness, reverse=True)
    return populacao[:len(populacao)//2]

In [20]:
def algoritmo_genetico(config: Config) -> list[int]:
    """
    Implementação do algoritmo genético clássico.

    Arguments:
        config: Parâmetros de configuração do AG.
    """

    # 1. t = 0
    t = 0
    
    # 2. Inicializar a população inicial P_0
    P = inicializar_populacao(config.tam_populacao, config.tam_cromossomo)
    
    # 3. Enquanto critério de parada == falso
    terminou = False
    while not terminou:
    
        # a. Avaliar a população(Pt)
        print('> Avaliação da população')
        for c in P:
            c.fitness = config.fitness(c)
            print(c)
        
        # b. P' = Selecionar pais(Pt)
        print('> Seleção dos pais')
        casais = config.selecionar_pais(P)
        for c in casais:
            print(c)
            
        # c. F = Aplicar recombinação e mutação(P')
        print('> Crossover e mutação')
        F = config.aplicar_cruzamento(casais)
        config.aplicar_mutacao(F, config.taxa_mutacao)
        
        # d. Avaliar a população(F)
        print('> Avaliação da população')
        for c in F:
            c.fitness = config.fitness(c)
            print(c)
        
        # e. Pt+1 = Selecionar sobreviventes(Pt + F)
        print('> Selecionar sobreviventes')
        P = config.selecionar_sobreviventes(P + F)
        for c in P:
            c.fitness = config.fitness(c)
            print(c)
            
        # f. t = t + 1
        t = t + 1
        break

## Teste do AG

In [21]:
# Problema de maximizar a quantidade de 1s no vetor

def maximizar_1s(cromossomo: Cromossomo) -> int:
    """
    Função de avaliação do problema. É nessa função que o cromossomo é interpretado
    para calcular a aptidão e verificar se está atingindo o objetivo.
    """
    return sum(cromossomo.dados)


# Parâmetros de configuração do AG
config = Config(
    tam_cromossomo=8,

    tam_populacao=10,
    fitness=maximizar_1s,

    taxa_mutacao=0.05,
    
    selecionar_pais=torneio,

    aplicar_cruzamento=crossover_1_corte,
    aplicar_mutacao=mutacao,
    selecionar_sobreviventes=elitismo,
)

# Execução do algoritmo genético
solucao = algoritmo_genetico(config)

> Avaliação da população
Cromossomo(dados=[0, 0, 1, 1, 1, 1, 1, 0], fitness=5)
Cromossomo(dados=[0, 1, 1, 1, 1, 1, 0, 1], fitness=6)
Cromossomo(dados=[1, 1, 0, 0, 1, 0, 0, 1], fitness=4)
Cromossomo(dados=[1, 0, 1, 1, 1, 0, 0, 0], fitness=4)
Cromossomo(dados=[1, 1, 1, 1, 0, 0, 0, 1], fitness=5)
Cromossomo(dados=[1, 1, 0, 1, 0, 0, 0, 1], fitness=4)
Cromossomo(dados=[1, 0, 0, 1, 1, 0, 1, 0], fitness=4)
Cromossomo(dados=[1, 1, 1, 0, 1, 0, 1, 0], fitness=5)
Cromossomo(dados=[1, 0, 1, 1, 0, 0, 1, 1], fitness=5)
Cromossomo(dados=[1, 1, 1, 1, 1, 1, 0, 1], fitness=7)
> Seleção dos pais
(Cromossomo(dados=[1, 1, 1, 1, 1, 1, 0, 1], fitness=7), Cromossomo(dados=[0, 0, 1, 1, 1, 1, 1, 0], fitness=5))
(Cromossomo(dados=[1, 1, 1, 1, 1, 1, 0, 1], fitness=7), Cromossomo(dados=[1, 0, 1, 1, 0, 0, 1, 1], fitness=5))
(Cromossomo(dados=[0, 0, 1, 1, 1, 1, 1, 0], fitness=5), Cromossomo(dados=[0, 1, 1, 1, 1, 1, 0, 1], fitness=6))
(Cromossomo(dados=[1, 1, 1, 1, 1, 1, 0, 1], fitness=7), Cromossomo(dados=[1, 1, 1, 